In [14]:
import pymongo
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import time

# Conectar a MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["OptiDataLake"]
collection = db["Analytics"]

# Widget para seleccionar la fecha de inicio y fin
date_picker_start = widgets.DatePicker(
    description='Fecha Inicio',
    disabled=False
)
date_picker_end = widgets.DatePicker(
    description='Fecha Fin',
    disabled=False
)

# Widgets para seleccionar criptomoneda y métrica para cada criptomoneda
crypto_selectors = {}
metric_selectors = {}
cryptos = ["BTCUSDT", "ETHUSDT", "XRPUSDT"]
metrics = ["volumen_total", "promedio_precios", "volatilidad", "movimiento_precios"]

# Crear widgets para cada criptomoneda
for crypto in cryptos:
    crypto_selectors[crypto] = widgets.Dropdown(
        options=metrics,
        value="volumen_total",
        description=f' {crypto}',
        disabled=False
    )

# Dropdown para la criptomoneda en la distribución de precios
price_distribution_crypto_dropdown = widgets.Dropdown(
    options=cryptos,
    value="BTCUSDT",
    description='Criptomoneda Precios',
    disabled=False
)

# Mostrar los widgets de selección de fecha, criptomoneda y métrica
display(date_picker_start, date_picker_end, price_distribution_crypto_dropdown)
for crypto in cryptos:
    display(crypto_selectors[crypto])

# Función para actualizar los gráficos
def update_graphs(change=None):
    if date_picker_start.value and date_picker_end.value:
        # Limpiar la salida anterior
        clear_output(wait=True)
        display(date_picker_start, date_picker_end, price_distribution_crypto_dropdown)
        for crypto in cryptos:
            display(crypto_selectors[crypto])
        
        # Convertir las fechas seleccionadas a formato compatible
        start_date = pd.to_datetime(date_picker_start.value).tz_localize('UTC')
        end_date = pd.to_datetime(date_picker_end.value).tz_localize('UTC')
        
        # Crear una figura para cada criptomoneda
        for crypto in cryptos:
            selected_metric = crypto_selectors[crypto].value
            # Obtener los datos filtrados por las fechas seleccionadas y criptomoneda
            data = list(collection.find({
                "time_window": {
                    "$gte": start_date,
                    "$lte": end_date
                },
                "cryptocurrency": crypto
            }))
            
            if data:
                df = pd.DataFrame(data)
                
                # Crear el gráfico para la criptomoneda seleccionada y la métrica seleccionada
                fig = go.Figure()
                fig.add_trace(go.Scatter(x=df['time_window'], y=df[selected_metric], mode='lines+markers',
                                         name=f'{selected_metric} {crypto}', line=dict(color="#636EFA")))

                fig.update_layout(
                    title=f'Visualización de {selected_metric} para {crypto}',
                    xaxis_title='Time Window',
                    yaxis_title='Valor',
                    template='plotly_white'
                )
                fig.show()
            else:
                print(f"No se encontraron datos para {crypto} en el rango de fechas seleccionado.")

        # Bubble Chart para Operaciones por Criptomoneda
        bubble_chart(start_date, end_date)

        # Gráfico de Distribución de Precios
        price_distribution_chart(start_date, end_date, price_distribution_crypto_dropdown.value)

# Registrar la función que se ejecutará cuando cambien los valores del DatePicker y los selectores
date_picker_start.observe(update_graphs, names='value')
date_picker_end.observe(update_graphs, names='value')
price_distribution_crypto_dropdown.observe(update_graphs, names='value')
for crypto in cryptos:
    crypto_selectors[crypto].observe(update_graphs, names='value')

# Bubble Chart para Operaciones por Criptomoneda
def bubble_chart(start_date, end_date):
    data = list(collection.find({
        "time_window": {
            "$gte": start_date,
            "$lte": end_date
        }
    }))
    
    if data:
        df = pd.DataFrame(data)
        df_grouped = df.groupby('cryptocurrency').agg({
            'volumen_total': 'sum',
            'promedio_precios': 'mean'
        }).reset_index()
        
        # Crear Bubble Chart
        fig = px.scatter(df_grouped, x='cryptocurrency', y='volumen_total', size='promedio_precios', color='cryptocurrency',
                         hover_name='cryptocurrency', size_max=60, title='Operaciones por Criptomoneda')
        fig.update_layout(template='plotly_white')
        fig.show()
    else:
        print("No se encontraron datos para el rango de fechas seleccionado.")

# Gráfico de Distribución de Precios
def price_distribution_chart(start_date, end_date, selected_crypto):
    # Obtener datos del rango de fechas seleccionado para la criptomoneda específica
    data = list(collection.find({
        "time_window": {
            "$gte": start_date,
            "$lte": end_date
        },
        "cryptocurrency": selected_crypto
    }))
    
    if data:
        df = pd.DataFrame(data)
        
        # Crear histograma de distribución de precios
        fig = px.histogram(df, x='promedio_precios', nbins=50, title=f'Distribución de Precios de {selected_crypto}')
        fig.update_layout(
            xaxis_title='Precio',
            yaxis_title='Frecuencia',
            template='plotly_white'
        )
        fig.show()
    else:
        print(f"No se encontraron datos para {selected_crypto} en el rango de fechas seleccionado.")

# Función para actualizar automáticamente cada 1 minuto
def auto_update():
    while True:
        time.sleep(60)  # Esperar 1 minuto
        update_graphs()

thread = threading.Thread(target=auto_update)
thread.daemon = True
thread.start()

DatePicker(value=None, description='Fecha Inicio', step=1)

DatePicker(value=None, description='Fecha Fin', step=1)

Dropdown(description='Criptomoneda Precios', options=('BTCUSDT', 'ETHUSDT', 'XRPUSDT'), value='BTCUSDT')

Dropdown(description=' BTCUSDT', options=('volumen_total', 'promedio_precios', 'volatilidad', 'movimiento_prec…

Dropdown(description=' ETHUSDT', options=('volumen_total', 'promedio_precios', 'volatilidad', 'movimiento_prec…

Dropdown(description=' XRPUSDT', options=('volumen_total', 'promedio_precios', 'volatilidad', 'movimiento_prec…

In [2]:
import pymongo
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import threading
import time
from flask import Flask, render_template_string, send_file, request
from pyngrok import ngrok
import io
import os
import plotly

# Conectar a MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["OptiDataLake"]
collection = db["Analytics"]

# Aplicación Flask
app = Flask(__name__)

# Template HTML básico para mostrar los gráficos
template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Dashboard Criptomonedas</title>
    <script>
        setInterval(function() {
            window.location.reload();
        }, 60000);
    </script>
</head>
<body>
    <h1>Dashboard Criptomonedas</h1>
    <div>
        <form action="/dashboard" method="get">
            {% for crypto in cryptos %}
            <label for="metric_{{ crypto }}">Métrica para {{ crypto }}:</label>
            <select name="metric_{{ crypto }}" id="metric_{{ crypto }}">
                <option value="volumen_total" {% if metrics[crypto] == 'volumen_total' %}selected{% endif %}>Volumen Total</option>
                <option value="promedio_precios" {% if metrics[crypto] == 'promedio_precios' %}selected{% endif %}>Promedio de Precios</option>
                <option value="volatilidad" {% if metrics[crypto] == 'volatilidad' %}selected{% endif %}>Volatilidad</option>
                <option value="movimiento_precios" {% if metrics[crypto] == 'movimiento_precios' %}selected{% endif %}>Movimiento de Precios</option>
            </select>
            <br>
            {% endfor %}
            <label for="price_crypto">Criptomoneda para Distribución de Precios:</label>
            <select name="price_crypto" id="price_crypto">
                {% for crypto in cryptos %}
                <option value="{{ crypto }}" {% if price_crypto == crypto %}selected{% endif %}>{{ crypto }}</option>
                {% endfor %}
            </select>
            <br>
            <label for="start">Fecha de Inicio:</label>
            <input type="date" id="start" name="start" value="{{ start }}">
            <label for="end">Fecha de Fin:</label>
            <input type="date" id="end" name="end" value="{{ end }}">
            <button type="submit">Generar Gráfico</button>
        </form>
    </div>
    {% if plots %}
    <div>
        {% for crypto, plot_div in plots.items() %}
        <h2>Gráfico de {{ metrics[crypto] }} para {{ crypto }}</h2>
        {{ plot_div|safe }}
        {% endfor %}
    </div>
    {% if price_plot %}
    <div>
        <h2>Distribución de Precios para {{ price_crypto }}</h2>
        {{ price_plot|safe }}
    </div>
    {% endif %}
    {% endif %}
</body>
</html>
"""

# Ruta para generar los gráficos y devolver el HTML embebido
def generate_plot(crypto, metric, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    data = list(collection.find({
        "time_window": {
            "$gte": start_date,
            "$lte": end_date
        },
        "cryptocurrency": crypto
    }))
    
    if data:
        df = pd.DataFrame(data)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df['time_window'], y=df[metric], mode='lines+markers',
                                 name=f'{metric} {crypto}', line=dict(color="#636EFA")))
        fig.update_layout(
            title=f'Visualización de {metric} para {crypto}',
            xaxis_title='Time Window',
            yaxis_title='Valor',
            template='plotly_white'
        )
        plot_div = plotly.offline.plot(fig, output_type='div')
        return plot_div
    return None

# Ruta para generar el gráfico de distribución de precios
def generate_price_distribution_plot(crypto, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    data = list(collection.find({
        "time_window": {
            "$gte": start_date,
            "$lte": end_date
        },
        "cryptocurrency": crypto
    }))
    
    if data:
        df = pd.DataFrame(data)
        fig = px.histogram(df, x='promedio_precios', nbins=50, title=f'Distribución de Precios para {crypto}')
        fig.update_layout(
            xaxis_title='Precio',
            yaxis_title='Frecuencia',
            template='plotly_white'
        )
        plot_div = plotly.offline.plot(fig, output_type='div')
        return plot_div
    return None

@app.route('/')
def home():
    # Mostrar una vista inicial sin gráficos
    return render_template_string(template, cryptos=["BTCUSDT", "ETHUSDT", "XRPUSDT"], metrics={}, price_crypto=None, start=None, end=None, plots=None, price_plot=None)

@app.route('/dashboard', methods=['GET'])
def dashboard():
    cryptos = ["BTCUSDT", "ETHUSDT", "XRPUSDT"]
    metrics = {}
    start = request.args.get('start')
    end = request.args.get('end')
    price_crypto = request.args.get('price_crypto')
    plots = {}
    price_plot = None

    for crypto in cryptos:
        metrics[crypto] = request.args.get(f'metric_{crypto}')

    if start and end:
        start_date = pd.to_datetime(start)
        end_date = pd.to_datetime(end)

        for crypto in cryptos:
            metric = metrics[crypto]
            if metric:
                plot_div = generate_plot(crypto, metric, start_date, end_date)
                if plot_div:
                    plots[crypto] = plot_div

        if price_crypto:
            price_plot = generate_price_distribution_plot(price_crypto, start_date, end_date)

        if plots or price_plot:
            return render_template_string(template, cryptos=cryptos, metrics=metrics, price_crypto=price_crypto, start=start, end=end, plots=plots, price_plot=price_plot)
    return render_template_string(template, cryptos=cryptos, metrics=metrics, price_crypto=price_crypto, start=start, end=end, plots=None, price_plot=None)

# Ejecutar el servidor Flask
if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print(f"Dashboard está corriendo en NgrokTunnel: {public_url}")
    app.run(host="0.0.0.0", port=5000, use_reloader=False)

Dashboard está corriendo en NgrokTunnel: NgrokTunnel: "https://4548-2800-300-9031-8810-b312-fa6c-5a8f-fb2d.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.53:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Oct/2024 20:09:35] "GET /dashboard?metric_BTCUSDT=volumen_total&metric_ETHUSDT=volumen_total&metric_XRPUSDT=volumen_total&price_crypto=BTCUSDT&start=2024-10-22&end=2024-10-24 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 20:10:37] "GET /dashboard?metric_BTCUSDT=volumen_total&metric_ETHUSDT=volumen_total&metric_XRPUSDT=volumen_total&price_crypto=BTCUSDT&start=2024-10-22&end=2024-10-24 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 20:11:40] "GET /dashboard?metric_BTCUSDT=volumen_total&metric_ETHUSDT=volumen_total&metric_XRPUSDT=volumen_total&price_crypto=BTCUSDT&start=2024-10-22&end=2024-10-24 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 20:12:42] "GET /dashboard?metric_BTCUSDT=volumen_total&metric_ETHUSDT=volumen_total&metric_XRPUSDT=volumen_total&price_crypto=BTCUSDT&start=2024-10-22&end=2024-10-24 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 2